In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'a') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [3]:
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

In [4]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [5]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [6]:
TEST_CASE_GENERATOR="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [7]:
def sanitize_string(data_str):
    # Replace single quotes with double quotes
    data_str = data_str.replace("'", '"')
    
    # Escape double quotes inside the values
    in_string = False
    sanitized_str = ""
    for i in range(len(data_str)):
        if data_str[i] == '"':
            if in_string and (i == 0 or data_str[i-1] != '\\'):
                sanitized_str += '\\"'
            else:
                sanitized_str += '"'
            in_string = not in_string if data_str[i] == '"' and (i == 0 or data_str[i-1] != '\\') else in_string
        else:
            sanitized_str += data_str[i]
    return sanitized_str

In [8]:
use_case_directory_path = r"D:\Dissertation-GPT\dataset\SpecificationData\Hotel Management System"
save_path = r"D:\Dissertation-GPT\evaluate\baseline 3 1.8\hotel"

In [9]:
usecase_scenario = {
    "Add booking": [
        "1. Successful Booking Addition",
        "2. Booking Addition with Missing Field Values",
        "3. Booking Addition with Invalid Field Values",
        "4. Booking Addition without Calculating Total Amount",
        "7. Booking Addition with Invalid Login Session"
    ],
    "Add Employee": [
        "1. Add Employee Successfully",
        "2. Add Employee with Missing Fields",
        "3. Add Employee with Invalid Field Data",
        "4. Add Employee with Duplicate Data"
    ],
    "Add Room": [
        "1. Add Room Successfully - Happy Path",
        "2. Add Room with Missing Field",
        "3. Add Room with Invalid Field Format",
        "4. Add Room with Duplicate Room Details",
        "6. Add Room and Then Cancel Before Submission"
    ],
    "Check in": [
        "1. Successful Check-In with Valid Booking ID",
        "4. Check-In with Already Used Booking ID",
        "5. Check-In with Booking ID of Another Guest",
        "6. Check-In without Entering Booking ID",
        "7. Check-In with Special Characters in Booking ID",
        "8. Check-In with SQL Injection in Booking ID Field",
        "9. Check-In with JavaScript Code in Booking ID Field",
        "10. Check-In with Extremely Long Booking ID"
    ],
    "Check out": [
        "1. Successful Checkout with Valid Booking ID",
        "2. Checkout Attempt with Invalid Booking ID",
        "4. Checkout Attempt with Previously Checked Out Booking ID"
    ],
    "Hotel Owner - Generate Reports": [
        "1. **Generate Report with Valid Parameters**: This scenario covers the happy path where the hotel owner successfully generates a report using valid parameters.",
        "2. **Generate Report with Invalid Parameters**: This scenario tests the system's response when the hotel owner tries to generate a report with invalid parameters, triggering the validation error.",
        "3. **Generate Report with Boundary Date Parameters**: This scenario checks the system's ability to handle reports generated with date parameters at the boundary of valid input (e.g., the earliest or latest possible dates).",
        "8. **Generate Report and Navigate Away Before Completion**: This scenario checks how the system handles situations where the user navigates away from the report generation page before the report is fully generated.",
        "9. **Generate Report with Special Characters in Parameters**: This scenario tests the system's handling of special characters in the report parameters, such as using symbols in date fields.",
        "12. **Generate Report with Future Date Parameters**: This scenario checks the system's response to attempts to generate a report for a future date range, which should not have any data.",
        "14. **Refresh Report Details Page**: This scenario tests the system's ability to refresh the report details page without errors or loss of data."
    ],
    "Hotel Owner - Update Hotel Information": [
        "1. Update Hotel Information Successfully",
        "2. Update Hotel Information with Invalid Data",
        "3. Update Hotel Information with Missing Mandatory Fields",
        "4. Update Hotel Information with Special Characters in Name",
        "5. Update Hotel Information with Extremely Long Values",
        "6. Update Hotel Information with SQL Injection Attempt",
        "7. Update Hotel Information with XSS Attack Attempt",
        "8. Update Hotel Information with No Changes Made"
    ],
    "login": [
        "1. Successful Login for Hotel Owner",
        "2. Successful Login for Manager",
        "3. Successful Login for Employee",
        "4. Login with Incorrect Credentials for Hotel Owner",
        "5. Login with Incorrect Credentials for Manager",
        "6. Login with Incorrect Credentials for Employee",
        "7. Login with Empty Username Field",
        "8. Login with Empty Password Field",
        "9. Login with Both Username and Password Fields Empty",
        "10. Login with Special Characters in Username",
        "11. Login with Special Characters in Password",
        "12. Login with SQL Injection Attack in Username Field",
        "13. Login with SQL Injection Attack in Password Field"
    ],
    "Request Service": [
        "1. Successful Room Service Request",
        "2. Room Service Request with Invalid Room ID",
        "3. Room Service Request with No Services Selected",
        "4. Room Service Request with Multiple Services Selected",
        "5. Room Service Request with Maximum Number of Services Selected"
    ],
    "Update Booking": [
        "1. Update Booking Details Successfully",
        "2. Update Booking with Non-Existent Booking ID",
        "3. Update Booking with Invalid Field Entries",
        "4. Update Booking Without Recomputing Payment for New Services",
        "6. Update Booking with All Fields Unchanged",
        "7. Update Booking with Some Fields Left Blank",
        "9. Update Booking with Maximum Field Length Values",
        "10. Update Booking with Minimum Field Length Values"
    ],
    "Update Employee": [
        "1. UpdateEmployee_SuccessfulUpdate",
        "2. UpdateEmployee_InvalidID",
        "3. UpdateEmployee_InvalidFieldEntry",
        "4. UpdateEmployee_NoChangesMade",
        "8. UpdateEmployee_UpdateWithSQLInjectionAttempt",
        "9. UpdateEmployee_UpdateWithXSSAttackAttempt",
        "10. UpdateEmployee_UpdateWithEmptyFields"
    ],
    "Update Room": [
        "1. Update Room Details Successfully Scenario",
        "2. Room Does Not Exist Scenario",
        "3. Update Room With Invalid Field Values Scenario",
        "4. Update Room With Missing Mandatory Fields Scenario",
        "6. Update Room Without Changes Scenario",
        "8. Update Room With Special Characters in Fields Scenario",
        "9. Update Room With Maximum Field Length Scenario",
        "10. Update Room With Minimum Field Length Scenario"
    ]
}

In [10]:
for key, value in usecase_scenario.items():
  usecase = key
  usecase_name =usecase
  usecase_path = os.path.join (use_case_directory_path,f"{usecase_name}.txt" )
  for testscenario in value:
    content = "Test scenarios:"+ testscenario+  "\nUse case:" + read_file_content(usecase_path)
    print(content)
    promptExtractCondition = [
      { "role": "system", "content": TEST_CASE_GENERATOR},
      { "role": "user", "content": content}
    ]
    gpt_response = askJSON(promptExtractCondition, client, model)
    print(gpt_response)
    print()
    json_data = json.loads(gpt_response)
    testscenario = testscenario.replace('"','').replace(':','').replace('?','').replace('*','').replace('/','')
    testscenario = testscenario[:150]
    testcase_path = os.path.join (save_path,f"{usecase_name}-{testscenario}.txt" )
    write_json_to_txt_raw(json_data,testcase_path )
  print("f{usecase_name} gen test case complete")

Test scenarios:1. Successful Booking Addition
Use case:4.8.1 Summary :
This use case is used to add bookings.
4.8.2 Actors :
Employee
4.8.3 Preconditions :
Employees must be logged in the system.
4.8.4  :
1. Actor Select Booking panel on dashboard.
2. System Show booking panel menu.
3. System Shows Booking fields
4. Actor Fills all the fields
5. Actor Click the calculate amount button to get total payable payment.
6. System Validate fields
7. System Generate Booking Id
8. Actor Press the add booking button.
9. System Validate fields
10. System Add booking to database
11. System Show initial status as checked in.
12. System Shows booking added.
4.8.5 Alternative path :
5. If the total amount is not calculated, bookingBasic course of events/happy path cannot be added.
6. If invalid field is entered, show invalid field entered for every individual error.
12. In case of any error show a failure message.
4.8.6 Post condition :
Bookings updated; Show the tables where all new booking is added